In [2]:
import cv2
import numpy as np

In [8]:
# language: python
import cv2
import numpy as np
import torch
from ultralytics import YOLO

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Cargar el modelo fine tuned (de YOLO_fine_tune.ipynb)
model = YOLO("../f1-strategy/weights/f1_detection_fine_tuned_best.pt")
model.to(DEVICE)

# Parámetros para el cálculo de gaps
CAR_LENGTH_METERS = 5.5  # Longitud del coche en metros
FRAME_WIDTH = 640        # Ancho del frame que se usará (redimensionar)

def calculate_gap(box1, box2, frame_width):
    """
    Calcula el gap entre dos coches basándose en sus bounding boxes.
    """
    x1, _, x2, _ = box1
    x3, _, x4, _ = box2
    center1 = (x1 + x2) / 2
    center2 = (x3 + x4) / 2
    pixel_distance = abs(center2 - center1)
    meters_per_pixel = CAR_LENGTH_METERS / (frame_width / 3)  # Conversión aproximada
    return pixel_distance * meters_per_pixel

def process_video_with_yolo(video_path):
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Redimensionar el frame para que coincida con el tamaño de entrada del modelo
        frame_resized = cv2.resize(frame, (FRAME_WIDTH, FRAME_WIDTH))
        
        # Obtener detecciones usando el método predict del modelo
        results = model.predict(frame_resized, conf=0.4, verbose=False)[0]
        
        # Verificar que se hayan detectado objetos
        if results.boxes is not None and len(results.boxes) > 0:
            # Extrae las bounding boxes y conviértelas a numpy array
            boxes = results.boxes.xyxy.cpu().numpy()  # Cada box: [x1, y1, x2, y2]
            
            # Ordenar las boxes según el centro en X
            boxes_sorted = sorted(boxes, key=lambda b: (b[0] + b[2]) / 2)
            
            # Dibujar cada bounding box
            for box in boxes_sorted:
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame_resized, (x1, y1), (x2, y2), (255, 0, 0), 2)
                
            # Calcular y dibujar el gap entre cada par de cajas consecutivas
            for i in range(len(boxes_sorted) - 1):
                gap = calculate_gap(boxes_sorted[i], boxes_sorted[i + 1], FRAME_WIDTH)
                
                # Calcular los centros horizontales de cada box
                b1 = boxes_sorted[i]
                b2 = boxes_sorted[i + 1]
                cx1 = int((b1[0] + b1[2]) // 2)
                cx2 = int((b2[0] + b2[2]) // 2)
                # Usar el valor de y inferior (de la caja superior) para anotar el gap
                y1_bottom = int(b1[3])
                
                # Dibujar la línea que une los centros
                cv2.line(frame_resized, (cx1, y1_bottom), (cx2, y1_bottom), (0, 255, 0), 2)
                
                # Escribir el gap (en metros) cerca de la línea
                cv2.putText(frame_resized, f"{gap:.2f}m", (cx1, y1_bottom - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        cv2.imshow("F1 Gap Detection", frame_resized)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

# Ejecutar detección de gaps en un video real
video_path = "../f1-strategy/data/videos/abu_dhabi_2024_race.mp4.f399.mp4"
process_video_with_yolo(video_path)